In [59]:
#===============
#   IMPORTS
#===============

# Fundamentals
import numpy as np
import pandas as pd

# Dash
import dash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input

# Dash in JupyterLab
from jupyter_dash import JupyterDash

# Plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from plotly.tools import mpl_to_plotly

pio.renderers.default = 'iframe'


#-STYLES----------------------------------
linestyles = ['-', '--', '-.', ':', (0, (5, 2, 1, 2, 1, 2))]
markers = ['o', 'x', 's', '^', 'd']
colors = ['#0077BB', '#33BBEE', '#009988', '#EE7733', '#CC3311', '#EE3377', '#BBBBBB']
colors4 = ['#0077BB', '#EE7733', '#CC3311', '#EE3377']
blues = ['#eff3ff','#c6dbef','#9ecae1','#6baed6','#3182bd','#08519c']
greys5 = ['#f7f7f7','#cccccc','#969696','#636363','#252525']
greys6 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']
greys7 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys8 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys9 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525','#000000']
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']


#====================
#   PY INCLUSIONS
#====================

# Data import
import baptistery_data_input as bdi
prism_data, levelling_data, extensimeter_data = bdi.readAllData()
prism_pos, levelling_pos, extensimeter_pos, positions = bdi.readSensorPositions()

def reformatPlot(fig, size=None, secondary=False):
    if size != None:
        fig.update_layout(height=size[1], width=size[0])
        
    fig.update_layout(xaxis_showline=True,
                      xaxis_mirror='ticks',
                      xaxis_linewidth=2,
                     yaxis_showline=True,
                     yaxis_mirror='ticks',
                     yaxis_linewidth=2)
    fig.update_layout(
        xaxis_ticks="inside",
        xaxis_ticklen=8,
        xaxis_tickwidth=2,
        yaxis_ticks="inside",
        yaxis_ticklen=8,
        yaxis_tickwidth=2
    )
    fig.update_layout(
        font_family="Roboto",
        hoverlabel_font_family="Roboto",
        xaxis_title_font_size=16,
        xaxis_tickfont_size=16,
        xaxis_showgrid=True,
        yaxis_title_font_size=16,
        yaxis_tickfont_size=16,
        yaxis_showgrid=True,
    )
    
    if secondary:    
        fig.update_layout(
                yaxis2=dict(
                title_font_size=16,
                tickfont_size=16,
                showgrid=False,
                showline=True,
                linewidth=2,
                ticks='inside',
                ticklen=8,
                tickwidth=2,
                title_font_color='gray',
                tickcolor='lightgray',
                tickfont_color='gray'
            ),
            yaxis_mirror=False
        )

    return fig

def selectPrismSection(n):
    """
    NOTE: for now, the function excludes prisms in the 
    1xx series.    
    """
    full_section = {
        '01':['01','07'],
        '02':['02','08'],
        '03':['03','09'],
        '04':['04','10'],
        '05':['05','11'],
        '06':['06','12'],
        '07':['07','01'],
        '08':['08','02'],
        '09':['09','03'],
        '10':['10','04'],
        '11':['11','05'],
        '12':['12','06']
    }
    n = full_section[n]
   
    selected_sensors = [p for p in prism_pos.index if p.endswith(n[0]) and not (p.startswith('1'))]
    selected_sensors += [p for p in prism_pos.index if p.endswith(n[1]) and not (p.startswith('1'))]
    
    return selected_sensors    


def rotTraslPrism(df, date):
    """
    Returns east and z coordinates of prisms
    in the DataFrame *df* for the date *date*.
    """
    x = df.loc[date, (slice(None), 'x')].values
    y = df.loc[date, (slice(None), 'y')].values
    z = df.loc[date, (slice(None), 'z')].values
    
    # Traslation
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    x1 = x - ref_x
    y1 = y - ref_y
    
    # Rotation
    a = -np.arctan(y1/x1)
    east = x1*np.cos(a) - y1*np.sin(a)
    north = x1*np.sin(a) + y1*np.cos(a)

    if (north > 10**(-6)).any():
        print("ERROR: north coordinate is not zero")
        return 1
    
    return east, z

def xyToEN(x, y):
    """
    Converts (x, y) coordinates of prisms to (E, N) ones.
    It takes either single or array-like coordinates.
    """
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    rot = np.deg2rad(37.1)
    e = x - ref_x
    n = y - ref_y
    eR = e*np.cos(rot) - n*np.sin(rot)
    nR = e*np.sin(rot) + n*np.cos(rot)
   
    return eR, nR

In [60]:
selectPrismSection('05')

['205', '305', '405', '505', '211', '311', '411', '511']

In [61]:
def figureSectionSelection(selected_prisms):
    """
    A small figure showing which section was selected.
    """
    fig = go.Figure(layout_template='plotly_white')
    fig.update_layout(height=250, width=250,
                         margin=dict(l=0,r=0,b=0,t=0))
    
    unselected_prisms = [el for el in prism_pos.index if (el not in selected_prisms and el[0] != '1')]
    up_x = [(a:=prism_pos.loc[i])['radius'] * np.cos(np.deg2rad(a['angle'])) 
            for i in unselected_prisms] 
    up_y = [(a:=prism_pos.loc[i])['radius'] * np.sin(np.deg2rad(a['angle'])) 
            for i in unselected_prisms]
    sp_x = [(a:=prism_pos.loc[i])['radius'] * np.cos(np.deg2rad(a['angle']))
            for i in selected_prisms] 
    sp_y = [(a:=prism_pos.loc[i])['radius'] * np.sin(np.deg2rad(a['angle']))
            for i in selected_prisms]
        
    # Add the shape of the Baptistery
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-17.80, y0=-17.80, x1=17.80, y1=17.80,
        line_color="lightgrey"
    )                
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-15.25, y0=-15.25, x1=15.25, y1=15.25,
        line_color="lightgrey",
        line_width=1
    )
    
    # Add unselected prisms
    fig.add_trace(
        go.Scatter(x=up_x, y=up_y,
                  mode='markers',
                  marker_color='lightblue',
                  hovertext=['Prism n. {}'.format(i) 
                             for i in unselected_prisms],
                  hoverinfo='text'
                  )
    )
    
    # Add selected prisms
    fig.add_trace(
        go.Scatter(x=sp_x, y=sp_y,
                  mode='markers',
                  marker_color='red',
                  hovertext=['Prism n. {}'.format(i) 
                             for i in selected_prisms],
                  hoverinfo='text'
                  )
    )
    
    # Disable the legend
    fig.update(layout_showlegend=False)
    # Format plot
    fig.update_layout(dict(
        xaxis_range = (-20, 20),
        yaxis_range = (-20, 20),
        xaxis_zeroline = False,
        xaxis_showgrid = False,
        yaxis_zeroline = False,
        yaxis_showgrid = False,
        xaxis_visible = False,
        yaxis_visible = False
    ))
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )
    
    return fig

In [45]:
def figureSectionRelativeDisplacements(prisms):
    """
    Produces a plot with the relative displacements of
    corresponding prisms in a section.
    """
    fig = go.Figure(layout_template=None)
    fig.update_layout(margin = dict(t=40, b=40))
    fig = make_subplots(specs=[[{"secondary_y": True}]],
                        figure=fig)
    fig = reformatPlot(fig, size=[950, 350], secondary=True)
    
    dists = np.sqrt(((prism_data[prisms[1]] - prism_data[prisms[0]])**2).sum(axis=1))*1000
    rel_disp = dists - dists[0]
    
    fig.add_trace(
        go.Scatter(
            x=prism_data.index,
            y=rel_disp,
            mode='markers+lines',
            name='Relative displacement',
            marker_color='#009988',
            line_color='#009988'
        ),
        secondary_y=False,
    )  
    
    fig.add_trace(
        go.Scatter(
            x=extensimeter_data.index,
            y=extensimeter_data['F4F8', 'temp'].rolling(24).mean(),
            line_dash='dot',
            line_color='gray',
            name='Temperature'
        ),
        secondary_y = True,
    )    
    
    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.02,
        y=0.95,
        text='<b>'+prisms[0] + '-' + prisms[1]+'</b>',
        font_family='Roboto',
        font_color='black',
        font_size=14,
        borderpad=4,
        bordercolor='black',
        borderwidth=1.5,
        showarrow=False
    )
    
    fig.update_yaxes(title_text="Displacement [mm]", secondary_y=False)
    fig.update_yaxes(title_text="Temperature [°C]", secondary_y=True)
    
    fig.update_layout(
        legend=dict(
            x=0.73, y=0.95
        )
    )
    
    return fig

def figurePrismCoupleSelection(selected_prisms):
    """
    A small figure showing which couple of prisms was selected.
    """
    fig = go.Figure(layout_template='plotly_white')
    fig.update_layout(height=250, width=250,
                         margin=dict(l=0,r=0,b=0,t=0))
    
    unselected_prisms = [el for el in prism_pos.index if (el not in selected_prisms and el[0] != '1')]
    up_x = [(a:=prism_pos.loc[i])['radius'] * np.cos(np.deg2rad(a['angle'])) 
            for i in unselected_prisms] 
    up_y = [(a:=prism_pos.loc[i])['radius'] * np.sin(np.deg2rad(a['angle'])) 
            for i in unselected_prisms]
    sp_x = [(a:=prism_pos.loc[i])['radius'] * np.cos(np.deg2rad(a['angle']))
            for i in selected_prisms] 
    sp_y = [(a:=prism_pos.loc[i])['radius'] * np.sin(np.deg2rad(a['angle']))
            for i in selected_prisms]
        
    # Add the shape of the Baptistery
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-17.80, y0=-17.80, x1=17.80, y1=17.80,
        line_color="lightgrey"
    )                
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-15.25, y0=-15.25, x1=15.25, y1=15.25,
        line_color="lightgrey",
        line_width=1
    )
    
    # Add unselected prisms
    fig.add_trace(
        go.Scatter(x=up_x, y=up_y,
                  mode='markers',
                  marker_color='lightblue',
                  hovertext=['Prism n. {}'.format(i) 
                             for i in unselected_prisms],
                  hoverinfo='text'
                  )
    )
    
    # Add selected prisms
    fig.add_trace(
        go.Scatter(x=sp_x, y=sp_y,
                  mode='markers+text',
                  text=selected_prisms,
                  textfont_family='Roboto',
                  textposition='top center',
                  textfont_size=12,
                  marker_color='red',
                  hovertext=['Prism n. {}'.format(i) 
                             for i in selected_prisms],
                  hoverinfo='text'
                  )
    )
    
    # Connecting line
    fig.add_shape(type="line",
            x0=sp_x[0], y0=sp_y[0], 
            x1=sp_x[1], y1=sp_y[1],
            line=dict(
                color='gray',
                width=2,
                dash='dash'
            ))
    
    # Disable the legend
    fig.update(layout_showlegend=False)
    # Format plot
    fig.update_layout(dict(
        xaxis_range = (-20, 20),
        yaxis_range = (-20, 20),
        xaxis_zeroline = False,
        xaxis_showgrid = False,
        yaxis_zeroline = False,
        yaxis_showgrid = False,
        xaxis_visible = False,
        yaxis_visible = False
    ))
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )
    
    return fig  

In [54]:
figurePrismCoupleSelection(['201', '207'])

In [74]:
ciccio = selectPrismSection('05')
ciccio.sort()
print(ciccio)

['205', '211', '305', '311', '405', '411', '505', '511']


In [71]:
def couplesFromSelectedPrisms(selected_prisms):
    """
    Returns couples of prisms, from a given selection,
    for which relative displacements should be plotted.    
    """
    # Rule 1: same beginnings
    